In [2]:
from torch.utils.data import DataLoader
from datasets import load_from_disk
from src.hyperdas.data_utils import generate_ravel_dataset, get_ravel_collate_fn, filter_dataset

from transformers import AutoTokenizer

%load_ext autoreload
%autoreload 2

/nlp/scr/sjd24/miniconda3/envs/hypernet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
tokenizer = AutoTokenizer.from_pretrained("/nlp/scr/sjd24/llama3-8b")

tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_dataset = load_from_disk("./experiments/RAVEL/data/nobel_prize_winner_train")
test_dataset = load_from_disk("./experiments/RAVEL/data/nobel_prize_winner_test")

collate_fn = get_ravel_collate_fn(tokenizer, add_space_before_target=True, contain_entity_position=True, source_suffix_visibility=True, base_suffix_visibility=True, bos_token_visibility=False)
dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=1, collate_fn=collate_fn, shuffle=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
test_dataset_erroneous_idxs = []
for i, d in enumerate(test_dataset):
    if d["target"] == "" or d["counterfactual_target"] == "":
        test_dataset_erroneous_idxs.append(i)

train_dataset_erroneous_idxs = []
for i, d in enumerate(train_dataset):
    if d["target"] == "" or d["counterfactual_target"] == "":
        train_dataset_erroneous_idxs.append(i)

In [20]:
train_dataset = [d for i, d in enumerate(train_dataset) if i not in train_dataset_erroneous_idxs]
test_dataset = [d for i, d in enumerate(test_dataset) if i not in test_dataset_erroneous_idxs]

from datasets import Dataset
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [21]:
train_dataset.save_to_disk("./experiments/RAVEL/data/nobel_prize_winner_train")
test_dataset.save_to_disk("./experiments/RAVEL/data/nobel_prize_winner_test")

Saving the dataset (1/1 shards): 100%|██████████| 14271/14271 [00:00<00:00, 377479.42 examples/s]
